In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import re
from random import shuffle
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
from googletrans import Translator
from unidecode import unidecode
from sklearn.model_selection import train_test_split
from random import shuffle

In [2]:
translator = Translator()

#Longest Word in English Language
maxLetters = 25

languageFamily = {
                "Germanic": '1000',
                "Romance": '0100',
                "Sino-Tebetan": '0010',
                "Japonic": '0001'
                }
"""Reads in all """
# Germanic Language Family
df = pd.read_csv("/Users/vincent/Git/PronunciationProject/NamePronunciationProject/ipa_dicts/Germanic/english-general_american.csv")
df.name = "English"
df2 = pd.read_csv("/Users/vincent/Git/PronunciationProject/NamePronunciationProject/ipa_dicts/Germanic/english-received_pronunciation.csv")
df2.name = "recevied"
df3 = pd.read_csv("/Users/vincent/Git/PronunciationProject/NamePronunciationProject/ipa_dicts/Germanic/german.csv")
df3.name = "german"
df4 = pd.read_csv("/Users/vincent/Git/PronunciationProject/NamePronunciationProject/ipa_dicts/Germanic/swedish.csv")
df4.name = "swed"
df5 = pd.read_csv("/Users/vincent/Git/PronunciationProject/NamePronunciationProject/ipa_dicts/Germanic/norwegian-bokmal.csv")
df5.name = "nor"

#Romance Language Family
df7 = pd.read_csv("/Users/vincent/Git/PronunciationProject/NamePronunciationProject/ipa_dicts/Romance/french-quebec.csv")
df7.name = "quebec"
df8 = pd.read_csv('/Users/vincent/Git/PronunciationProject/NamePronunciationProject/ipa_dicts/Romance/spanish-mexico.csv')
df8.name = "mexico"


#Sino-Tebetan Family

#Cantonese is having an issue reading cantonese
df10 = pd.read_csv('/Users/adamvalencia/Git/NamePronunciationProject/ipa_dicts/Sino-Tebetan/cantonese.csv')
df10.name = "cantonese"

#Mandarin has two spellings per word sometimes. Have to standardize data.
df11 = pd.read_csv('/Users/vincent/Git/PronunciationProject/NamePronunciationProject/ipa_dicts/Sino-Tebetan/mandarin.csv')
df11.name = "mandarin"

#Japonic
df12 = pd.read_csv('/Users/vincent/Git/PronunciationProject/NamePronunciationProject/ipa_dicts/japanese.csv')
df12.name = "japonic"

datasets = [df, df2, df3, df4, df5, df7, df8, df10, df11, df12]

def removeIpaSlash(s):
  """Function to remove slashses from the IPA Pronunciation"""
  return s.replace("/", "")

cleanDatasets = []
for d in datasets:
    d['Pronunciations'] = d['Pronunciations'].map(removeIpaSlash)
    cleanDatasets.append(d.sample(frac=1).reset_index(drop=True))
    cleanDatasets[len(cleanDatasets) - 1].name = d.name

languageTags = {
            "Germanic": [cleanDatasets[0],cleanDatasets[1],cleanDatasets[2],cleanDatasets[3],cleanDatasets[4]],
            "Romance": [cleanDatasets[5],cleanDatasets[6]],
            "Sino-Tebetan":[cleanDatasets[7], cleanDatasets[8]],
            "Japonic": [cleanDatasets[9]]
            }


/opt/homebrew/Caskroom/miniforge/base/envs/ENG_FAM/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [10]:
# def processWord(word, tag):
#     """Process a string that removes punctuation and is not longer than 15 letters."""
#     uniEncoded = []
#     print("Orginal is: " + word)
#     if tag == "Sino-Tebetan":
#         lower = translator.translate(word, dest='en', src='zh-cn').text.lower()
#     elif tag == "Japonic":
#         lower = translator.translate(word, dest='en',  src='ja').text.lower()
#     else:
#         lower = word.lower()
    
#     finalString = re.sub(r'[^A-Za-zÀ-ÖØ-öø-ÿ]', '', lower)
#     print("translated word is: " + finalString)
#     if len(finalString) <= maxLetters:
#         # uniEncoded.append(finalString)
#         return finalString

#     return ''

# def unicodeConverter(char):
#     """Returns the array ascii representation for a word
#     There are 52 total lettesr in ASCII"""
#     uniValue = ord(char)

#     #Ranges for ASCII Represneation of word
#     if uniValue in range(ord('A'), ord('Z') + 1):
#         return uniValue - ord('A')
#     elif uniValue in range(ord('a'),ord('z') + 1):
#         return uniValue - ord('a')
#     elif uniValue in range(ord('À'),ord('Ö') + 1):
#         return uniValue - ord('À')
#     elif uniValue in range(ord('Ø'),ord('ö') + 1):
#         return uniValue - ord('Ø')
#     elif uniValue in range(ord('ø'),ord('ÿ') + 1):
#         return uniValue - ord('ø')
#     elif uniValue in range(ord('Ø'),ord('ö') + 1):
#         return uniValue - ord('Ø')
#     else:
#         print("Unrecognized Character " + char)
#         exit(1)

def convertDictionaryToVector(word):
    """Given a word convert the string to their binary
    vector counterpart."""

    vector = ""
    for letter in word:
        ind = unicodeConverter(letter)
        vector += (str(0)*ind) + str(1) + str(0)*(51-ind)
    if len(word) <= maxLetters:
        vector += str(0)*52*(maxLetters - len(word))
    else:
        vector = ""
    
    return vector

def createOutputVector(tagIdx, numLanguages):
    return str(0)*tagIdx + str(1) + str(0)*(numLanguages - 1 - tagIdx)


In [4]:
languageTags["Germanic"][2]

,Word,Pronunciations,AltPronunciations1,AltPronunciations2,AltPronunciations3
0,täuschest,ˈtɔøʃɛst,NaN,NaN,NaN
1,Münzen,ˈmʏnt͡sn̩,NaN,NaN,NaN
2,zusammenbrechendes,ˈtsuːˌzammɛnbrɛçɛndəs,NaN,NaN,NaN
3,Püppchens,ˈpʏppçɛns,NaN,NaN,NaN
4,dezentere,deˈtsɛntɐ̯ə,NaN,NaN,NaN
...,...,...,...,...,...
278909,gottgegebenem,ˈgɔttgegebənəm,NaN,NaN,NaN
278910,pendelndes,ˈpɛndɛlndəs,NaN,NaN,NaN
278911,schärferem,ˈʃɛɾfɐ̯əm,NaN,NaN,NaN
278912,anzurechnend,ʔanˈtsuːˌɾɛçnɛnt,NaN,NaN,NaN


In [5]:
temparr = []
for dataset in languageTags["Germanic"]:
    sampleSetPopulation = len(dataset)//3
    print(dataset.name)
    for i in range(sampleSetPopulation):
#         print(i)
        temp = []
        binaryV = convertDictionaryToVector(processWord(dataset['Word'][i], "Germanic"))
        for j in binaryV:
            temp.append(int(j))
        for k in languageFamily["Germanic"]:
            temp.append(int(k))
        temparr.append(temp)

print(len(temparr))
bigdataset = pd.DataFrame(temparr)

English
recevied
german
swed
nor
167068


In [6]:
bigdataset

,0,1,2,3,4,5,6,7,8,9,...,1294,1295,1296,1297,1298,1299,1300,1301,1302,1303
0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167063,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
167064,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
167065,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
167066,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [7]:
temparr2 = []
for dataset in languageTags["Romance"]:
    sampleSetPopulation = len(dataset)//3
    print(dataset.name)
    for i in range(sampleSetPopulation):
#         print(i)
        temp = []
        binaryV = convertDictionaryToVector(processWord(dataset['Word'][i], "Romance"))
        for j in binaryV:
            temp.append(int(j))
        for k in languageFamily["Romance"]:
            temp.append(int(k))
        temparr2.append(temp)

print(len(temparr2))
bigdataset2 = pd.DataFrame(temparr2)

########SPANISH##########
# temparr5 = []
# for i in range(len(df8['binaryVector'])):
#     temp = []
#     for j in df8['binaryVector'][i]:
#         temp.append(int(j))
#     for k in df8['languageFamily'][i]:
#         temp.append(int(k))
#     temparr5.append(temp)

# samplePopulation5 = shuffle(temparr5)
# bigdataset2 = pd.DataFrame(temparr5[:len(temparr5)//3])

quebec
mexico
280614


In [8]:
# #######FRENCH###########
# temparr2 = []
# for i in range(len(df7['binaryVector'])):
#     temp = []
#     for j in processWord df7['Word'][i]:
#         temp.append(int(j))
#     for k in df7['languageFamily'][i]:
#         temp.append(int(k))
#     temparr2.append(temp)

# samplePopulation2 = shuffle(temparr2)
# bigdataset3 = pd.DataFrame(temparr2[:len(temparr2)//3])

In [17]:
# # translator.raise_Exception = True
# temparr3 = []
# for dataset in languageTags["Sino-Tebetan"]:
#     sampleSetPopulation = len(dataset)//3
#     print(sampleSetPopulation)
#     print(dataset.name)
#     for i in range(20):
#         print(i)
#         temp = []
#         binaryV = convertDictionaryToVector(processWord(dataset['Word'][i], "Sino-Tebetan"))
#         for j in binaryV:
#             temp.append(int(j))
#         for k in languageFamily["Sino-Tebetan"]:
#             temp.append(int(k))
#         temparr3.append(temp)

# print(len(temparr3))
# bigdataset3 = pd.DataFrame(temparr3)

# # ##################
# # samplePopulation3 = shuffle(temparr3)
# # bigdataset4 = pd.DataFrame(temparr3[:len(temparr3)//3])
# # .sample(frac=1)

# # temparr3 = []
# # for  i in range(1304):
# #     temp = []
# #     binaryV = convertDictionaryToVector(procesWord(dataset['Word'][i], "Sino-Tebetan"))
# #     for j in binaryV:
# #         temp.append(int(j))
# #     for k in dataset['languageFamily'][i]:
# #             temp.append(int(k))
# #     temparr3.append(temp)

# # samplePopulation3 = shuffle(temparr3)
# # bigdataset4 = pd.DataFrame(temparr3[:len(temparr3)//3])

14926
mandarin
0
Orginal is: 行事
translated word is: act
1
Orginal is: 特技
translated word is: trick
2
Orginal is: 驴驹
translated word is: foal
3
Orginal is: 放鞭炮
translated word is: firecracker
4
Orginal is: 栨
translated word is: bamboo
5
Orginal is: 前门
translated word is: frontdoor
6
Orginal is: 稗史
translated word is: bookofanecdotes
7
Orginal is: 洋裙
translated word is: skirt
8
Orginal is: 拱手
translated word is: gong
9
Orginal is: 檐沟
translated word is: pitch
10
Orginal is: 梈
translated word is: bamboo
11
Orginal is: 干瘦
translated word is: skinny
12
Orginal is: 心弦
translated word is: heartstring
13
Orginal is: 㼮
translated word is: bamboo
14
Orginal is: 自背黑锅
translated word is: backbone
15
Orginal is: 违约
translated word is: default
16
Orginal is: 婆婆
translated word is: motherinlaw
17
Orginal is: 拉扯
translated word is: pull
18
Orginal is: 辞世
translated word is: resign
19
Orginal is: 复印
translated word is: copy
20


In [ ]:
# temparr4 = []
# for dataset in languageTags["Japonic"]:
#     sampleSetPopulation = len(dataset)//3
#     print(dataset.name)
#     for i in range(sampleSetPopulation):
# #         print(i)
#         temp = []
#         binaryV = convertDictionaryToVector(processWord(dataset['Word'][i], "Japonic"))
#         for j in binaryV:
#             temp.append(int(j))
#         for k in languageFamily["Japonic"]:
#             temp.append(int(k))
#         temparr4.append(temp)

# print(len(temparr))
# bigdataset4 = pd.DataFrame(temparr4)

# ##################
# # temparr4 = []
# # for dataset in languageTags["Japonic"]:
# #     for  i in range(len(dataset['binaryVector'])):
# #         temp = []
# #         binaryV = convertDictionaryToVector(procesWord(dataset['Word'][i], "Japonic"))
# #         for j in binaryV:
# #             temp.append(int(j))
# #         for k in dataset['languageFamily'][i]:
# #             temp.append(int(k))
# #         temparr4.append(temp)

# # samplePopulation4 = shuffle(temparr4)
# # bigdataset5 = pd.DataFrame(temparr4[:len(temparr4)//3])

In [ ]:
# shuffledDatasets = [bigdataset, bigdataset2, bigdataset3, bigdataset4]
# concatenatedDatasets = pd.concat(shuffledDatasets)
shuffledDatasets = [bigdataset, bigdataset2]
concatenatedDatasets = pd.concat(shuffledDatasets)

In [ ]:
# %store concatenatedDatasets

In [ ]:
# finalDataset = concatenatedDatasets.sample(frac=1)

In [ ]:
network = Sequential()
network.add(Dense(150, activation='relu'))
network.add(Dense(100, activation='relu'))
network.add(Dense(100, activation='relu'))
network.add(Dense(2, activation='sigmoid'))
network.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# # print(len(resultXTrain))
# # print(len(resultYTrain))
# # print(len(resultXTest))
# # print(len(resultYTest))

In [ ]:
datasetLength = len(concatenatedDatasets)
index = round(datasetLength * .05)
print(index)


N = 4
# Select last 4 columns of dataframe
xValues = concatenatedDatasets.iloc[:, 0:-N]
yValues = concatenatedDatasets.iloc[: , -N:]
resultXTrain = xValues[:index]
resultXTest = xValues[index:]
resultYTrain = yValues[:index]
resultYTest = yValues[index:]
print("X Values :")
print(xValues)
print("Y Values :")
print(yValues)

# %store resultXTrain
# %store resultYTrain
# %store resultXTest
# %store resultYTest

In [ ]:
# for val in finalDataset[0][0]:
#     print(type(val))

In [ ]:
# resultXTrain
# print(resultXTrain)
# print(type(resultXTrain))

In [ ]:
# finalDataset

In [ ]:
# resultYTrain
# print(resultYTrain)
# print(type(resultYTrain))

In [ ]:
# network.fit(resultXTrain, resultYTrain, epochs=80, validation_data=(resultXTest, resultYTest), verbose=2)

In [ ]:
# NN = Sequential()
# NN.add(Dense(150, activation='relu'))
# NN.add(Dense(100, activation='relu'))
# NN.add(Dense(100, activation='relu'))
# NN.add(Dense(len(languageTags), activation='sigmoid'))
# NN.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# NN.fit(resultXTrain, resultYTrain, epochs=85, validation_data=(resultXTest, resultYTest), verbose=2)

In [ ]:
# words = ["colonel", "Garçon", "對唔住", "おねがいします"]
# processedWords = [processWord(words[0], "Germanic"),
#                  processWord(words[1], "Romance"),
#                  processWord(words[2], "Sino-Tebetan"),
#                  processWord(words[3], "Japonic")]
# print("\n\n\n\n\n")
# # for v in processedWords:
# #     print(v)

# # print(translator.translate(words[2], dest='en', src='zh-cn').text.lower())
# # print(translator.translate("おねがいします", dest='en',src='ja').text.lower())